### Import Required Libraries and Set Up Environment Variables

In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# Prints API key to ensure it is loaded correctly
print(os.getenv("NASA_API_KEY")) 

z3afBLJBtzh2onHmXA0KJ88TWtfVkzfEnCARzbg7


### CME Data

In [8]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url_CME = f"https://api.nasa.gov/DONKI/CME?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [14]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url_CME)

In [15]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [16]:
# Preview ONLY the first element from the cme_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[0], indent=4))

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [18]:
# Convert cme_json to a Pandas DataFrame 
# # Keep only the columns: activityID, startTime, linkedEvents
cme_df = pd.DataFrame(cme_json)[["activityID", "startTime", "linkedEvents"]]


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [20]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset=["linkedEvents"], inplace=True)


,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [ ]:
# Initialize an empty list to store the expanded rows
expanded_rows = []

# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element
# Iterate over each index in the DataFrame
# Iterate over each dictionary in the list
# Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
# Create a new DataFrame from the expanded rows

for i in cme_df.index:
    activity_id = cme_df.loc[i, "activityID"]
    start_time = cme_df.loc[i, "startTime"]
    linked_events = cme_df.loc[i, "linkedEvents"]

    for event in linked_events:
        expanded_rows.append({
            "cmeID": activity_id,
            "startTime": start_time,
            "linkedEvent": event
        })

cme_expanded_df = pd.DataFrame(expanded_rows)

In [27]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
 # Log the error or print it for debugging

def extract_activityID_from_dict(input_dict):
        try:
                return input_dict.get("ActivityID", None)
        except (ValueError, TypeError, AttributeError):
                return None


print(cme_expanded_df.columns)

Index(['cmeID', 'startTime', 'linkedEvent'], dtype='object')


In [28]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_expanded_df["GST_ActivityID"] = cme_expanded_df["linkedEvent"].apply(extract_activityID_from_dict)

In [29]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_expanded_df.dropna(subset=["GST_ActivityID"], inplace=True)

In [34]:
# print out the datatype of each column in this DataFrame:
print(cme_expanded_df.dtypes)


cmeID                     object
startTime_CME     datetime64[ns]
GST_ActivityID            object
dtype: object
Index(['cmeID', 'startTime_CME', 'GST_ActivityID'], dtype='object')


In [39]:
# Convert the 'GST_ActivityID' column to string format 
cme_expanded_df["GST_ActivityID"] = cme_expanded_df["GST_ActivityID"].astype(str)

# Convert startTime to datetime format
cme_expanded_df.rename(columns={"startTime_CME": "startTime_CME", "cmeID": "cmeID"}, inplace=True)

# Rename startTime_CME to startTime and activityID to cmeID
cme_expanded_df.rename(columns={"startTime_CME": "startTime", "activityID": "cmeID"}, inplace=True)

# Drop linkedEvents
# dropped above when we created the expanded_rows list; breaks code otherwise

# Verify that all steps were executed correctly
print(cme_expanded_df.head())
print(cme_expanded_df.dtypes)

cmeID                     object
startTime         datetime64[ns]
GST_ActivityID            object
dtype: object


In [40]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_expanded_df = cme_expanded_df[cme_expanded_df["GST_ActivityID"].str.contains("GST")]

In [41]:
# Adding final check
print(f"Final CME dataframe shape: {cme_expanded_df.shape}")
print(cme_expanded_df.head())

Final CME dataframe shape: (0, 3)
Empty DataFrame
Columns: [cmeID, startTime, GST_ActivityID]
Index: []


### GST Data

In [42]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_GST = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [43]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)

In [44]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

In [45]:
# Preview ONLY the first element from the gst_json list you created in JSON format
# Do NOT print out the entire list
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[0], indent=4))

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [48]:
# Convert gst_json to a Pandas DataFrame
# Keep only the columns: gstID, startTime, linkedEvents
gst_df = pd.DataFrame(gst_json)[["gstID", "startTime", "linkedEvents"]]  


In [49]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df.dropna(subset=["linkedEvents"], inplace=True)

In [50]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode("linkedEvents").reset_index(drop=True)


In [51]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df["CME_ActivityID"] = gst_df["linkedEvents"].apply(extract_activityID_from_dict)

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df.dropna(subset=["CME_ActivityID"], inplace=True)


In [ ]:
# Convert the 'CME_ActivityID' column to string format 

# Convert the 'gstID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_GST 

# Drop linkedEvents

# Verify that all steps were executed correctly


In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  


### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.


In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst


### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


### Exporting data in csv format

In [28]:
# Export data to CSV without the index
